# 100 - ChEMBL data description

## Introduction
Throughout this notebook, we will explore the ChEMBL database. We will look at the number of rows, the type of data available and the SQL statements needed to extract the required data.  We will do some cleaning too, if this is required.

In [1]:
# global variables

import mysql.connector
from mysql.connector import errorcode

sc.setLogLevel("INFO")

config = {
  'user': 'joseph',
  'password': 'password',
  'host': '192.168.151.11',
  'database': 'chembl_22',
  'raise_on_warnings': True,
}

## Small Molecules
We need the following data
- molregno: this is an integer used to identify a molecule
- canonical_smiles: a varchar(4000) identifying the molecule in SMILES format

NOTE: All data is filtered using Taxonomy ID = 9606 -> Homo Sapiens

In [2]:
# Get all molecules from ChEMBL and store them in a SPARK dataframe
molecules = None   # a dataframe to store small molecules

try:    
    cnx = mysql.connector.connect(**config)
  
    cursor = cnx.cursor()

    query = ("SELECT DISTINCTROW cs.molregno, cs.canonical_smiles " +
             "FROM compound_structures cs, activities act, " +
             "     assays asy " +
             "WHERE cs.molregno = act.molregno AND" +
             "     act.assay_id = asy.assay_id AND" +
             "     asy.assay_tax_id = 9606")

    cursor.execute(query)

    tempMolecules = []
    i = 0
    for (molregno, canonical_smiles) in cursor:
        i = i + 1
        tempMolecules.append((molregno, canonical_smiles))
        if i % 10000 == 0:
            print i
            if molecules is None:
                molecules = spark.createDataFrame(tempMolecules)
            else:
                _molecules = spark.createDataFrame(tempMolecules)
                molecules = molecules.union(_molecules)
            tempMolecules = []

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cnx.close()
    
# store the rest  
if len(tempMolecules) > 0:
    _molecules = spark.createDataFrame(tempMolecules)
    molecules = molecules.union(_molecules)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000


In [4]:
# store molecules in hadoop hdfs
print("Storing %d molecules in hdfs" % molecules.count())
molecules.coalesce(1).write.csv("hdfs://hadoop1:9000/ics5200/molecules.csv")

Storing 991054 molecules in hdfs


## Target components
Target proteins in ChEMBL are stored under the components group of tables.  Next, we will get the component ID and sequence for each component in ChEMBL.  Sequence is the FASTA sequence.

In [9]:
# Get all proteins from ChEMBL and store them in a SPARK dataframe
proteins = None   # a dataframe to store proteins

try:
    cnx = mysql.connector.connect(**config)

    cursor = cnx.cursor()

    query = ("SELECT DISTINCTROW component_id, sequence " +
             "FROM component_sequences " +
             "WHERE tax_id = 9606")

    cursor.execute(query)

    tempProteins = []
    i = 0
    for (component_id, sequence) in cursor:
        i = i + 1
        tempProteins.append((component_id, sequence))
        if i % 1000 == 0:
            print i
            if proteins is None:
                proteins = spark.createDataFrame(tempProteins)
            else:
                _proteins = spark.createDataFrame(tempProteins)
                proteins = proteins.union(_proteins)
            tempProteins = []

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cnx.close()

# store the rest  
if len(tempProteins) > 0:
    _proteins = spark.createDataFrame(tempProteins)
    proteins = proteins.union(_proteins)

1000
2000
3000


In [10]:
# store proteins in hadoop hdfs
print("Storing %d proteins in hdfs" % proteins.count())
proteins.coalesce(1).write.csv("hdfs://hadoop1:9000/ics5200/proteins.csv")

Storing 3130 proteins in hdfs


## Ligand-Protein bindings
Bindings in ChEMBL are extracted from activities and assays.  we store the following data:
- ASSAY_ID: Unique ID for the assay
- MOLREGNO: Unique compound (moleucule) ID
- STANDARD_RELATION: Symbol constraining the activity value (e.g. >, <, =)
- STANDARD_VALUE: Datapoint value from published source but transformed to common units: e.g. mM concentrations converted to nM.
- STANDARD_UNITS: Selected 'Standard' units for data type: e.g. concentrations are in nM.
- STANDARD_TYPE: Standardised version of the published_activity_type (e.g. IC50 rather than Ic-50/Ic50/ic50/ic-50)
- PCHEMBL_VALUE: Negative log of selected concentration-response activity values (IC50/EC50/XC50/AC50/Ki/Kd/Potency)

**NOTE:** We shall cast decimal values as in MYSQL Decimal values are stored as DECIMAL(64,30), which is not supported in Python.  Altough this will generate a number of 1264 Warings, this is not affecting our final result, as ChEMBL values are up to two significat figures (https://www.ebi.ac.uk/chembl/faq):

In addition to the conversion of published activity types/values/units to standard activity types/values/units, described in previous releases, a number of further enhancements have been made to the data in the activities table: 

Conversion of log/-log values to nM concentrations. For example pIC50 and log Ki have been converted to IC50 and Ki values.
Rounding of standard values to three significant figures (or 2 decimal places for values > 10)
Flagging of data with possible errors (using the data_validity_comment field), such as unusual units for the activity type, or very large/small numbers.
Identification of potential duplicate values - where an activity measurement is likely to be a repeat citation of an earlier measurement, rather than an independent measurement (flagged using the potential_duplicate column).

An additional table (ACTIVITY_STDS_LOOKUP) has been included in this release, which contains details of the standard_types that have been standardised, their permitted standard_units, and acceptable value ranges.

``+---------+------+-------------------------------------------------------------------------------------+
| Level   | Code | Message                                                                             |
+---------+------+-------------------------------------------------------------------------------------+
| Warning | 1264 | Out of range value for column 'CAST(act.standard_value as DECIMAL(38,30))' at row 1 |
| Warning | 1264 | Out of range value for column 'CAST(act.standard_value as DECIMAL(38,30))' at row 1 |
|....                                                                                                  |
+---------+------+-------------------------------------------------------------------------------------+``

In [11]:
# Get all proteins from ChEMBL and store them in a SPARK dataframe
bindings = None   # a dataframe to store proteins

try:
    # some data is null, thus Spark cannot infer the schema
    # assay_id, molregno, std_relation, std_value, std_units, std_type, pchembl_value, component_id
    # this schema is infered after anlysing the schema using rdd.printSchema()
    from pyspark.sql.types import *
    bindingSchema = StructType([StructField("assay_id", LongType(), True),
                                StructField("molregno", LongType(), True),
                                StructField("std_relation", StringType(), True),
                                StructField("std_value", DecimalType(38,30), True),
                                StructField("std_units", StringType(), True),
                                StructField("std_type", StringType(), True),
                                StructField("pchembl_value", DecimalType(38,30), True),
                                StructField("component_id", LongType(), True)])

    cnx = mysql.connector.connect(**config)

    cursor = cnx.cursor()

    query = ("SELECT DISTINCTROW act.assay_id, act.molregno, " +
             "     act.standard_relation, CAST(act.standard_value as DECIMAL(38,30)), " +
             "     act.standard_units, act.standard_type, " +
             "     CAST(act.pchembl_value as DECIMAL(38,30)), tc.component_id " +
             "FROM activities act, assays asy,  " +
             "     target_components tc "
             "WHERE act.assay_id = asy.assay_id AND " +
             "      asy.tid = tc.tid AND "+ 
             "      asy.assay_tax_id = 9606")

    cursor.execute(query)

    tempBindings = []
    i = 0
    for (assay_id, molregno, std_relation, std_value, std_units, std_type, pchembl_value, component_id) in cursor:
        i = i + 1
        tempBindings.append((assay_id, molregno, std_relation, std_value, std_units, std_type, pchembl_value, component_id))
        if i % 100000 == 0:
            print i
            if bindings is None:
                bindings = spark.createDataFrame(tempBindings, bindingSchema)
            else:
                _bindings = spark.createDataFrame(tempBindings, bindingSchema)
                bindings = bindings.union(_bindings)
            tempBindings = []
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    elif err.errno == errorcode.ER_WARN_DATA_OUT_OF_RANGE:
        print("There were Data out of range warnings, but this is ok.  See note in Jupyter Notebook.")
    else:
        print(err)
else:
    cnx.close()

# store the rest  
if len(tempBindings) > 0:
    _bindings = spark.createDataFrame(tempBindings, bindingSchema)
    bindings = bindings.union(_bindings)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
There were Data out of range warnings, but this is ok.  See note in Jupyter Notebook.


In [ ]:
#### bindings.printSchema()

In [ ]:
# store brindings in hadoop hdfs
#print("Storing %d bindings in hdfs" % bindings.count())
bindings.write.csv("hdfs://hadoop1:9000/ics5200/bindings.csv")